# What are Decorators

Decorators are annotations that we apply to classes, methods, properties, or parameters to modify how they behave. They’re widely used in frameworks like Angular and Vue.

TypeScript itself doesn’t ship with built‑in decorators, so we create our own. At runtime, a decorator is simply a function that the JavaScript engine invokes.

Because decorators are still an experimental feature, you must enable them in the TypeScript compiler before using them—set the `experimentalDecorators` option to `true` in your `tsconfig.json`.

# Class Decorators

Class decorators receive only one parameter—the class constructor function

In [ ]:
function Component(constructor:Function){   // decorator receives the class constructor (not an instance)
    console.log("Component decorator called");
    constructor.prototype.uniqueId = Date.now();
    constructor.prototype.insertInDom = () => {
        console.log("Inserting component into the DOM");
    }
}

@Component
class ProfileComponent{
}

Component decorator called


**No matter how many instances** of the class you create (0, 1, or 10), the **class decorator** gets called **only once**, when the class is **defined**, not when it's instantiated.

Note(Extra): We can achieve similar behavior using inheritance—for example, by creating a base class like `Component` and having `ProfileComponent` extend it. So Decorators are another tool in our toolBox.

# Parameterized Decorators

In [5]:
// Define a type for the options that can be passed to the decorator
type ComponentOptions = {
    selector: string;
}

// This is a decorator factory – it takes options and returns the actual decorator function
function Component(options : ComponentOptions) {
    // This is the actual decorator function that receives the class constructor
    return (constructor: Function) => {
        console.log("Component decorator called");
        constructor.prototype.options = options;
        constructor.prototype.uniqueId = Date.now();
        constructor.prototype.insertInDom = () => {
            console.log("Inserting component into the DOM");
        }
    }
}

// Apply the decorator to the class with custom options
@Component({selector: '#app-profile'})
class ProfileComponent {
}

Component decorator called


[Function (anonymous)]

# Decorator Compisition

We can apply multiple decorators to a class or its members

In [7]:
type ComponentOptions = {
    selector: string;
}

function Component(options : ComponentOptions) {
    return (constructor: Function) => {
        console.log("Component decorator called");
        constructor.prototype.options = options;
        constructor.prototype.uniqueId = Date.now();
        constructor.prototype.insertInDom = () => {
            console.log("Inserting component into the DOM");
        }
    }
}

function Pipe(constructor : Function){
    console.log("Pipe decorator called");
    constructor.prototype.pipe = true;
}


@Component({selector: '#app-profile'})
@Pipe
class ProfileComponent {
}

Pipe decorator called
Component decorator called


[Function (anonymous)]

In the example above, decorators are called in reverse order. This idea comes from math, like in F(g(x)) — where g(x) is called first, and then its result is passed to F.

Similarly, here the Pipe decorator is called first, and then the result is passed to the Component decorator.

# Method Decorators

Method decorators receive three parameters: the target object, the name of the method, and the property descriptor.

In [ ]:
function Log(target : any, methodName: string, descriptor : PropertyDescriptor) {
    const original = descriptor.value as Function;
    descriptor.value = function() {
        console.log('Before')
        original.call(this, 'Blue Sky');
        console.log('After')
    }
}

class Person {
    @Log
    say(message: string) {
        console.log('Person says:'+ message);
    }
}

const person = new Person();
person.say('Hello World'); 

In the example above, the output will be:

- Before
- Person says: Blue Sky
- After

The value we pass when calling `person.say('Hello World')` is **ignored** because it is replaced by the hardcoded value `'Blue Sky'` inside the decorator.

This happens because in the decorator, we replace the original `say` method with a new implementation that always calls the original method with `'Blue Sky'` as the argument, instead of using the argument passed (`'Hello World'`).

So, the original parameter `message` is not used in the new implementation inside the decorator.


If we want to use the actual argument passed to the method (like `'Hello World'`), we can update the decorator like this:

In [ ]:
function Log(target : any, methodName: string, descriptor : PropertyDescriptor) {
    const original = descriptor.value as Function;
    descriptor.value = function(message: string) {
        console.log('Before')
        original.call(this, message);
        console.log('After')
    }
}

class Person {
    @Log
    say(message: string) {
        console.log('Person says:'+ message);
    }
}

const person = new Person();
person.say('Hello World'); 

Now , the output will be:

- Before
- Person says: Hello World
- After

But in the previous version, the decorator only works well when the method accepts one specific type of parameter. If we want the decorator to be more flexible and work with **any method and any number of parameters**, we can write it like this:

In [ ]:
function Log(target : any, methodName: string, descriptor : PropertyDescriptor) {
    const original = descriptor.value as Function;
    descriptor.value = function(...args: any) {
        console.log('Before')
        original.call(this, ...args);
        console.log('After')
    }
}

class Person {
    @Log
    say(message: string) {
        console.log('Person says:'+ message);
    }

    @Log
    sayWithGreeting(greeting: string, message: string) {
        console.log(`${greeting}, ${message}`);
    }
}

const person = new Person();
person.say('Hello World'); 
person.sayWithGreeting('Hi', 'everyone!');

Now , the output will be:

- Before
- Person says: Hello World
- After
- Before
- Hi, everyone!
- After

# Access Decorators

We can create decorators that can be applied to getters and setters as well.

In [ ]:
function Capitalize(target:any, methodName: string, descriptor: PropertyDescriptor) {
   const original = descriptor.get;
   descriptor.get = function() {
         const result = original?.call(this);
        if (typeof result === 'string') {
            return result.toUpperCase();
        }
        return result;
    }
}


class Person {
    constructor(public firstName: string, public lastName: string) {}

    @Capitalize
    get fullName() {
        return `${this.firstName} ${this.lastName}`;
    }
}


let person1 = new Person('John', 'Doe');
console.log(person1.fullName); // Outputs: JOHN DOE

# Property Decorator

In [ ]:
// A property decorator that enforces a minimum string length
function MinLength(length: number) {
     // This function returns a decorator function
    return (target:any, propertyName: string) => {
        let value: string ;

         // Define a custom property descriptor with getter and setter
        const descriptor : PropertyDescriptor = {
            get() {
                // Return the stored value
                return value;
            },
            set(newValue : string) {
                // Check if the new value meets the minimum length
                if(newValue.length < length){
                    throw new Error(`${propertyName} should be at least ${length} characters long`);
                }
                value = newValue;
            }
        }
        // Override the default property with the custom getter and setter
        Object.defineProperty(target, propertyName, descriptor);
    }
}

class User {
    // This decorator will enforce that the password must be at least 4 characters long
    @MinLength(4)
    password: string;

    constructor(password: string) {
        this.password = password; // This will trigger the custom setter
    }
}

let user = new User('1234'); 
console.log(user.password); // Outputs: 1234

# Parameter Decorator

Parameter decorators are not something we use often in everyday code. However, they can be very useful if you're building a framework or library for other developers to use.

In [ ]:
// Define a type to store info about watched parameters
type WatchedParameter = {
    methodName: string;
    parameterIndex: number;
}

// Global array to keep track of all watched parameters
const watchedParameters: WatchedParameter[] = [];

// Parameter decorator function
function Watch(target: any, methodName: string, parameterIndex: number){
    // Store metadata about the watched parameter
    watchedParameters.push({
        methodName,
        parameterIndex
    });
}

class Vehicle {
    move(@Watch speed:number) {}
}

console.log(watchedParameters); // Outputs: [{ methodName: 'move', parameterIndex: 0 }]